In [74]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('data_final.csv')

In [3]:
df.shape

(37790, 6)

In [4]:
df.head()

,ID commande,Nom du fabricant,Product Category Default,Article,Code postal (Facture),Ville (Facture)
0,1743,FERME BIXARTEA,CRÈMERIE,breuil (375g).,64250.0,SAINT PALAIS
1,1743,FERME BIXARTEA,FROMAGES,fromage de brebis fermier (650g environ) +6 mo...,64250.0,SAINT PALAIS
2,1743,FERME AROZTEGIA,FRUITS & LÉGUMES,radis noir (la botte),64250.0,SAINT PALAIS
3,1743,FERME ESNEKIAK,CRÈMERIE,Yaourt nature bio au lait entier (4 x125 gr).,64250.0,SAINT PALAIS
4,1743,FERME BELAZKABIETA,ÉPICERIE,BIZIGAI Piment d'Espelette AOP (50 gr),64250.0,SAINT PALAIS


## 1 Modification du nom des colonnes

In [5]:
df.columns

Index(['ID commande', 'Nom du fabricant', 'Product Category Default',
       'Article', 'Code postal (Facture)', 'Ville (Facture)'],
      dtype='object')

In [6]:
df.columns = ['commande','producteur','categorie','nom_produit','code_postal','commune']

## 2 Modification contenu code_postal

In [7]:
df.code_postal.dtypes

dtype('float64')

In [8]:
df.code_postal = df.code_postal.astype(int)

In [9]:
df.dtypes

commande        int64
producteur     object
categorie      object
nom_produit    object
code_postal     int64
commune        object
dtype: object

In [10]:
df.head()

,commande,producteur,categorie,nom_produit,code_postal,commune
0,1743,FERME BIXARTEA,CRÈMERIE,breuil (375g).,64250,SAINT PALAIS
1,1743,FERME BIXARTEA,FROMAGES,fromage de brebis fermier (650g environ) +6 mo...,64250,SAINT PALAIS
2,1743,FERME AROZTEGIA,FRUITS & LÉGUMES,radis noir (la botte),64250,SAINT PALAIS
3,1743,FERME ESNEKIAK,CRÈMERIE,Yaourt nature bio au lait entier (4 x125 gr).,64250,SAINT PALAIS
4,1743,FERME BELAZKABIETA,ÉPICERIE,BIZIGAI Piment d'Espelette AOP (50 gr),64250,SAINT PALAIS


In [11]:
df.code_postal.unique()

array([64250, 64240, 64780, 64480, 64640, 64100, 64990, 64600, 64310,
       64210, 64200, 64120, 40220, 64122, 40440, 77700, 64340, 35580,
       40530, 31300, 24270, 64500, 38000, 64330, 64190, 64140, 64220,
       79400, 40300, 64300, 78000, 64259, 40390])

In [12]:
len(df.commande.unique())

5270

In [13]:
df.shape

(37790, 6)

In [14]:

len(df.nom_produit.unique())

311

In [15]:
df[df.categorie == 'FROMAGES']['nom_produit'].unique()

array(["fromage de brebis fermier (650g environ) +6 mois d'affinage.",
       'Fromage Pérail de brebis.',
       "fromage de brebis fermier (1.5kg environ) +6 mois d'affinage.",
       'Sarrin frais.', 'Espil.', 'Garralda.', 'Ixuri.', 'Xedarri.',
       'Patin.',
       "fromage de brebis fermier (1.5 kg environ) -6 mois d'affinage.",
       "fromage de brebis fermier (650g environ) -6 mois d'affinage.",
       "fromage de brebis fermier (500g environ) 10/12 mois d'affinage.",
       "fromage de brebis fermier (1kg environ) 10/12 mois d'affinage.",
       "fromage de brebis fermier (3KG) - 6 mois d'affinage.",
       'Etxebazter.', 'Manex.'], dtype=object)

In [16]:
len(df[df.categorie == 'FROMAGES']['nom_produit'].unique())

16

In [17]:
for cat in df.categorie.unique():
    for produit in df[df.categorie == cat]['nom_produit'].unique():
        print(cat,produit)

CRÈMERIE breuil (375g).
CRÈMERIE Yaourt nature bio au lait entier (4 x125 gr).
CRÈMERIE Fromage blanc de vache nature,lisse bio (500g).
CRÈMERIE Yaourt a la fraise bio au lait entier  (4 x 125 gr).
CRÈMERIE Beurre doux bio au lait cru ( 250 gr).
CRÈMERIE Creme fraiche bio  au lait cru (250ml).
CRÈMERIE Lait de vache demi-ecreme pasteurise bio (1L).
CRÈMERIE Yaourt a la framboise bio  au lait entier (4 x125 gr).
CRÈMERIE Fromage blanc de vache nature campagne bio (500g).
CRÈMERIE Yaourt a la myrtille bio  au lait entier (4 x 125g).
CRÈMERIE Breuil (1kg).
CRÈMERIE Lait de vache entier pasteurise bio (1L).
CRÈMERIE Mamia (x4).
CRÈMERIE Breuil (500g).
CRÈMERIE Yaourt au citron, bio,  au lait entier (4 x 125g).
CRÈMERIE Beurre salé bio au lait cru (250g).
CRÈMERIE Yaourt a la vanille  bio  au lait entier ( 4 x 125 gr).
CRÈMERIE Yaourt brebis nature (lot de 4).
CRÈMERIE Yaourt brebis fruits rouges coulis (lot de 4).
CRÈMERIE Yaourt aux fruits, bio, au lait entier (4 x 125g).
CRÈMERIE Yaourt 

## 3 Reduction du nombre de produits
Actuellement il y a 311 produits différents, mais on remarque que l'on peut réunir en un seul plusieurs produits (par exemple il y a 7 "fromages de brebis fermier" car avec des poids différents...)

### Pour la catégorie CRÈMERIE

In [18]:
print(df[df.categorie == 'CRÈMERIE']['nom_produit'].unique(), len(df[df.categorie == 'CRÈMERIE']['nom_produit'].unique()))

['breuil (375g).' 'Yaourt nature bio au lait entier (4 x125 gr).'
 'Fromage blanc de vache nature,lisse bio (500g).'
 'Yaourt a la fraise bio au lait entier  (4 x 125 gr).'
 'Beurre doux bio au lait cru ( 250 gr).'
 'Creme fraiche bio  au lait cru (250ml).'
 'Lait de vache demi-ecreme pasteurise bio (1L).'
 'Yaourt a la framboise bio  au lait entier (4 x125 gr).'
 'Fromage blanc de vache nature campagne bio (500g).'
 'Yaourt a la myrtille bio  au lait entier (4 x 125g).' 'Breuil (1kg).'
 'Lait de vache entier pasteurise bio (1L).' 'Mamia (x4).'
 'Breuil (500g).' 'Yaourt au citron, bio,  au lait entier (4 x 125g).'
 'Beurre salé bio au lait cru (250g).'
 'Yaourt a la vanille  bio  au lait entier ( 4 x 125 gr).'
 'Yaourt brebis nature (lot de 4).'
 'Yaourt brebis fruits rouges coulis (lot de 4).'
 'Yaourt aux fruits, bio, au lait entier (4 x 125g).'
 'Yaourt brebis citron (lot de 4).' 'Lait de chevre AB (1L).'
 'Fromage blanc de chevre (500g).' 'Fromage blanc de chevre (420g).'] 24


On peut réduire le nombre actuels de produits de 24 à 7
- breuil
- mamia
- yaourt
- fromage blanc
- lait
- beurre
- creme fraiche

In [19]:
def create_product(nom):
    nom = nom.lower()
    if 'yaourt' in nom:
        return 'yaourt'
    elif 'breuil' in nom:
        return 'breuil'
    elif 'mamia' in nom:
        return 'mamia'
    elif 'fromage blanc' in nom:
        return 'fromage blanc'
    elif 'beurre' in nom:
        return 'beurre'
    elif 'creme fraiche' in nom:
        return 'creme fraiche'
    elif 'lait' in nom:
        return 'lait'

df['produit'] = df[df.categorie == 'CRÈMERIE']['nom_produit'].apply(create_product)

In [20]:
df[df.categorie == 'CRÈMERIE']['produit'].unique()

array(['breuil', 'yaourt', 'fromage blanc', 'beurre', 'creme fraiche',
       'lait', 'mamia'], dtype=object)

### Catégorie Fromages et Fromage de vache
Pour simplifier nous réunissons le traitement de ces deux catégories

In [21]:
df.categorie.unique()

array(['CRÈMERIE', 'FROMAGES', 'FRUITS & LÉGUMES', 'ÉPICERIE', 'FRUITS',
       'VOLAILLES & OEUFS', 'VIANDES', 'VOLAILLES', 'ÉPICERIE SUCRÉE',
       'YAOURT & FROMAGE BLANC', 'VIANDE FRAÎCHE', 'ÉPICERIE SALÉE',
       'LÉGUMES', 'BOISSONS', 'NOS PRODUITS', 'FROMAGE DE VACHE',
       'PLATS CUISINÉS'], dtype=object)

In [22]:
filtre = (df.categorie == 'FROMAGES') | (df.categorie == 'FROMAGE DE VACHE')

In [23]:
df[filtre]

,commande,producteur,categorie,nom_produit,code_postal,commune,produit
1,1743,FERME BIXARTEA,FROMAGES,fromage de brebis fermier (650g environ) +6 mo...,64250,SAINT PALAIS,NaN
16,1750,FERME BIXARTEA,FROMAGES,Fromage Pérail de brebis.,64240,HASPARREN,NaN
21,1751,FERME BIXARTEA,FROMAGES,Fromage Pérail de brebis.,64240,AYHERRE,NaN
41,1754,FERME BIXARTEA,FROMAGES,fromage de brebis fermier (650g environ) +6 mo...,64240,hasparren,NaN
73,1758,FERME BIXARTEA,FROMAGES,Fromage Pérail de brebis.,64240,hasparren,NaN
...,...,...,...,...,...,...,...
37765,7203,FERME IXURIBEHEREA,FROMAGES,Garralda.,64240,ayherre,NaN
37766,7203,FERME BIXARTEA,FROMAGES,fromage de brebis fermier (650g environ) +6 mo...,64240,ayherre,NaN
37767,7203,FERME OHETA,FROMAGE DE VACHE,"Fromage de vache affiné (4-6 mois) 1,1 kg.",64240,ayherre,NaN
37779,7204,FERME BIXARTEA,FROMAGES,fromage de brebis fermier (650g environ) +6 mo...,64480,Ustaritz,NaN


In [24]:
def create_fromage(nom):
    nom = nom.lower()
    if 'fromage de brebis fermier' in nom:
        return 'fromage de brebis fermier'
    elif 'fromage pérail de brebis' in nom:
        return 'fromage pérail de brebis'
    elif 'tomme de vache fraîche' in nom:
        return 'tomme de vache fraîche'
    elif 'fromage de vache affiné' in nom:
        return 'fromage de vache affiné'
    else:
        return 'fromage de chèvre'


df['produit'] = df[filtre]['nom_produit'].apply(create_fromage)

/home/jean/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


### Catégorie Fruits & légumes et catégorie fruits
Pour simplifier nous réunissons le traitement de ces deux catégories également

In [87]:
filtre2 = (df.categorie == 'FRUITS & LÉGUMES') | (df.categorie == 'FRUITS')

In [101]:
df_fruits = df[filtre2].copy()

In [102]:
df_fruits['nom_produit'].unique()

array(['radis noir (la botte)', 'betterave rouge (la botte)',
       'Pommes Gordin Xuri (1kg)', 'courge potimarron ou butternut (1kg)',
       'courge longue de naples (1kg)',
       "Pomme de terre de l'année(2.5kg)", 'Persil (la botte)',
       'batavia ou feuille de chene',
       'assortiment chou fleur / romanesco (0,8-1kg)', 'Mache AB (200gr)',
       'chou frise ou cabus (600gr a 800gr)', 'Epinard (300g)',
       'chicoree scarole', 'choux de Bruxelles (500g)',
       'pomme de terre vapeur (2kg)', 'navet (750g)',
       "Pomme de terre de l'année (1kg)", 'chou de chine',
       'Cagette "surprise"', "Poireaux d'HIVER (lot de 3 ou 4)",
       'Fenouil (500gr)',
       'Céleri branche (le bouquet de 5 ou 6 branches)',
       'Oignon  rouge ( la botte)', 'Ail sec (300gr)',
       'Blette (la botte)', 'feves (1-1,2kg)', 'petit pois Mangetout',
       'pomme de terre primeur 1kg', 'Cerises Bigarreau (600 gr)',
       'Cerises Bigarreau (300 gr)', 'radis de printemps (la botte)',
  

In [193]:
len(df_fruits['nom_produit'].unique())

89

In [205]:
def product_fruits_legumes(row):
    nom = row.nom_produit
    nom = nom.lower()
    if 'radis' in nom:
        return 'radis'
    elif 'betterave' in nom:
        return 'betterave'
    elif 'pomme de terre' in nom:
        return 'pomme de terre'
    elif 'pommes' in nom or 'sagarra' in nom:
        return 'pommes'
    elif 'persil' in nom:
        return 'persil'
    elif ('batavia' in nom)|('mache' in nom)|('scarole' in nom)|('roquette' in nom) or 'mesclun' in nom:
        return 'salade'
    elif 'chou' in nom:
        return 'chou'
    elif 'oignon' in nom:
        return 'oignon'
    elif 'courge' in nom or 'musquee de provence' in nom:
        return 'courge'
    elif 'epinard' in nom:
        return 'epinard'
    elif 'navet' in nom:
        return 'navet'
    elif 'cagette' in nom:
        return 'assortiment'
    elif 'poireaux' in nom:
        return 'poireaux'
    elif 'fenouil' in nom:
        return 'fenouil'
    elif 'blette' in nom:
        return 'blette'
    elif 'ail sec' in nom or 'aillet' in nom:
        return 'celeri'
    elif 'céleri' in nom or 'celeri' in nom:
        return 'celeri'
    elif 'feves' in nom:
        return 'feves'
    elif 'petit pois' in nom:
        return 'petit pois'
    elif 'cerises' in nom:
        return 'cerises'
    elif 'agrumes' in nom or 'clementines' in nom or 'oranges' in nom or 'citron' in nom or 'pomelos' in nom:
        return 'agrumes'
    elif 'piment' in nom:
        return 'piment'
    elif 'concombre' in nom:
        return 'concombre'
    elif 'carotte' in nom:
        return 'carotte'
    elif 'tomate' in nom:
        return 'tomate'
    elif 'aubergine' in nom:
        return 'aubergine'
    elif 'kiwi' in nom:
        return 'kiwi'
    elif 'echalote' in nom:
        return 'echalote'
    elif 'poivron' in nom:
        return 'poivron'
    else:
        return 'error'



df_fruits['produit'] = df_fruits.apply(product_fruits_legumes,axis=1)

In [206]:
len(df_fruits['produit'].unique())

29

In [207]:
df_fruits['produit'].unique()

array(['radis', 'betterave', 'pommes', 'courge', 'pomme de terre',
       'persil', 'salade', 'chou', 'epinard', 'navet', 'assortiment',
       'poireaux', 'fenouil', 'celeri', 'oignon', 'blette', 'feves',
       'petit pois', 'cerises', 'piment', 'concombre', 'tomate',
       'aubergine', 'echalote', 'carotte', 'agrumes', 'kiwi', 'poivron',
       'error'], dtype=object)